In [5]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

In [6]:
data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


As this is a problem of natural language processing, I’ll start by tokenizing the data

In [7]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

Now we need to pad the sequences to the same length to feed them into a neural network.  pad the sequences of the texts to have the same length

In [8]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

use the label encoder method to convert the classes from strings to a numerical representation:

In [9]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [10]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [11]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, 
                                                one_hot_labels, 
                                                test_size=0.2)

In [12]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
400/400 [==============================] - 19s 45ms/step - loss: 1.3513 - accuracy: 0.4812 - val_loss: 0.9112 - val_accuracy: 0.6722
Epoch 2/10
400/400 [==============================] - 18s 44ms/step - loss: 0.3576 - accuracy: 0.8862 - val_loss: 0.5211 - val_accuracy: 0.8338
Epoch 3/10
400/400 [==============================] - 21s 51ms/step - loss: 0.0637 - accuracy: 0.9823 - val_loss: 0.5821 - val_accuracy: 0.8269
Epoch 4/10
400/400 [==============================] - 15s 38ms/step - loss: 0.0271 - accuracy: 0.9941 - val_loss: 0.5887 - val_accuracy: 0.8316
Epoch 5/10
400/400 [==============================] - 15s 37ms/step - loss: 0.0190 - accuracy: 0.9959 - val_loss: 0.6375 - val_accuracy: 0.8281
Epoch 6/10
400/400 [==============================] - 15s 37ms/step - loss: 0.0131 - accuracy: 0.9975 - val_loss: 0.6527 - val_accuracy: 0.8269
Epoch 7/10
400/400 [==============================] - 15s 37ms/step - loss: 0.0113 - accuracy: 0.9979 - val_loss: 0.6864 - val_accuracy:

 take a sentence as an input text and see how the model performs:

In [13]:
input_text = "She didn't come today because she lost her dog yestertay!"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 [==============================] - 0s 126ms/step
['sadness']


Possibility to optimize that ML , accuracy kinda weak but before i will try each ML technique to understand how it works